In [1]:
import requests as rq
from bs4 import BeautifulSoup

# 네이버 금융에서 이전 2영업일에 해당하는 날짜 추출
url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)

  |  2024.05.08


In [2]:
import re

# 영업일 양식 변경 및 추출
biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)
print(biz_day)

20240508


In [3]:
import requests as rq
from io import BytesIO
import pandas as pd

# OTP 생성
gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'mktId': 'STK',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

print(otp_stk)

OlEtruLAhPYJYqb+UZPDxgXm2+ecE4KGjjo9MxFU2f4RtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15efSXMRnkMufaBu1zuWfTzostBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


In [4]:
# OTP 제출 및 증시 데이터 가져오기
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = rq.post(down_url, {'code': otp_stk}, headers=headers)
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')

sector_stk.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,서비스업,4830,5,0.10,218570825970
1,006840,AK홀딩스,KOSPI,기타금융,15390,-90,-0.58,203879963790
2,027410,BGF,KOSPI,기타금융,3745,45,1.22,358459382295
3,282330,BGF리테일,KOSPI,유통업,136200,2700,2.02,2354067997200
4,138930,BNK금융지주,KOSPI,기타금융,8520,60,0.71,2744193491760


In [5]:
gen_otp_ksq = {
    'mktId': 'KSQ',  # 코스닥 입력
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

down_sector_ksq = rq.post(down_url, {'code': otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR ')

sector_ksq.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,2665,-25,-0.93,129350150930
1,054620,APS,KOSDAQ,금융,7100,-110,-1.53,141248969100
2,265520,AP시스템,KOSDAQ,반도체,34900,1100,3.25,533321592900
3,211270,AP위성,KOSDAQ,통신장비,18000,-500,-2.70,271481472000
4,109960,AP헬스케어,KOSDAQ,유통,623,-2,-0.32,71116867948


In [6]:
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)
krx_sector['종목명'] = krx_sector['종목명'].str.strip()
krx_sector['기준일'] = biz_day

krx_sector.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,서비스업,4830,5,0.10,218570825970,20240508
1,006840,AK홀딩스,KOSPI,기타금융,15390,-90,-0.58,203879963790,20240508
2,027410,BGF,KOSPI,기타금융,3745,45,1.22,358459382295,20240508
3,282330,BGF리테일,KOSPI,유통업,136200,2700,2.02,2354067997200,20240508
4,138930,BNK금융지주,KOSPI,기타금융,8520,60,0.71,2744193491760,20240508


In [7]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType': '1',
    'mktId': 'ALL',
    'trdDd': biz_day,
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = rq.post(down_url, {'code': otp}, headers=headers)

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
krx_ind['기준일'] = biz_day

krx_ind.head()

,종목코드,종목명,종가,대비,등락률,EPS,PER,BPS,PBR,주당배당금,배당수익률,기준일
0,060310,3S,2665,-25,-0.93,30.0,88.83,947.0,2.81,0,0.00,20240508
1,095570,AJ네트웍스,4830,5,0.10,367.0,13.16,9326.0,0.52,270,5.59,20240508
2,006840,AK홀딩스,15390,-90,-0.58,2635.0,5.84,44339.0,0.35,200,1.30,20240508
3,054620,APS,7100,-110,-1.53,667.0,10.64,11683.0,0.61,0,0.00,20240508
4,265520,AP시스템,34900,1100,3.25,3997.0,8.73,21396.0,1.63,270,0.77,20240508


In [8]:
diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))
print(diff)

['맵스리얼티1', 'ESR켄달스퀘어리츠', '코람코더원리츠', '한국패러랠', '엑세스바이오', '한국ANKOR유전', '엘브이엠씨홀딩스', '신한알파리츠', 'JTC', '미래에셋맵스리츠', '디앤디플랫폼리츠', '제이알글로벌리츠', 'NH프라임리츠', 'KB스타리츠', '소마젠', '윙입푸드', '맥쿼리인프라', '크리스탈신소재', '오가닉티코스메틱', '골든센츄리', 'NH올원리츠', '마스턴프리미어리츠', 'SK리츠', '코람코라이프인프라리츠', '이스트아시아홀딩스', '잉글우드랩', '프레스티지바이오파마', 'GRT', '이지스밸류리츠', '미래에셋글로벌리츠', '케이탑리츠', '에이리츠', '로스웰', '씨엑스아이', 'SBI핀테크솔루션즈', '이리츠코크렙', '컬러레이', '네오이뮨텍', '삼성FN리츠', '스타리츠', '롯데리츠', '글로벌에스엠', '헝셩그룹', '한화리츠', '애머릿지', '코오롱티슈진', '고스트스튜디오', '신한서부티엔디리츠', '이지스레지던스리츠']


In [9]:
kor_ticker = pd.merge(krx_sector,
                      krx_ind,
                      on=krx_sector.columns.intersection(
                          krx_ind.columns).tolist(),
                      how='outer')

kor_ticker.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일,EPS,PER,BPS,PBR,주당배당금,배당수익률
0,095570,AJ네트웍스,KOSPI,서비스업,4830,5,0.10,218570825970,20240508,367.0,13.16,9326.0,0.52,270.0,5.59
1,006840,AK홀딩스,KOSPI,기타금융,15390,-90,-0.58,203879963790,20240508,2635.0,5.84,44339.0,0.35,200.0,1.30
2,027410,BGF,KOSPI,기타금융,3745,45,1.22,358459382295,20240508,813.0,4.61,17286.0,0.22,120.0,3.20
3,282330,BGF리테일,KOSPI,유통업,136200,2700,2.02,2354067997200,20240508,11337.0,12.01,62197.0,2.19,4100.0,3.01
4,138930,BNK금융지주,KOSPI,기타금융,8520,60,0.71,2744193491760,20240508,1905.0,4.47,31746.0,0.27,510.0,5.99


In [10]:
import numpy as np

kor_ticker['종목구분'] = np.where(kor_ticker['종목명'].str.contains('스팩|제[0-9]+호'), '스팩',
                              np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                                       np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠',
                                                np.where(kor_ticker['종목명'].isin(diff),  '기타',
                                                '보통주'))))
kor_ticker = kor_ticker.reset_index(drop=True)
kor_ticker.columns = kor_ticker.columns.str.replace(' ', '')
kor_ticker = kor_ticker[['종목코드', '종목명', '시장구분', '종가',
                         '시가총액', '기준일', 'EPS', 'BPS', '주당배당금', '종목구분']]
kor_ticker = kor_ticker.replace({np.nan: None})
kor_ticker['기준일'] = pd.to_datetime(kor_ticker['기준일'])

kor_ticker.head()

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,BPS,주당배당금,종목구분
0,095570,AJ네트웍스,KOSPI,4830,218570825970,2024-05-08,367.0,9326.0,270.0,보통주
1,006840,AK홀딩스,KOSPI,15390,203879963790,2024-05-08,2635.0,44339.0,200.0,보통주
2,027410,BGF,KOSPI,3745,358459382295,2024-05-08,813.0,17286.0,120.0,보통주
3,282330,BGF리테일,KOSPI,136200,2354067997200,2024-05-08,11337.0,62197.0,4100.0,보통주
4,138930,BNK금융지주,KOSPI,8520,2744193491760,2024-05-08,1905.0,31746.0,510.0,보통주


fnguide에서 추출하는 데이터 내용을 아래와 같음

| 순서 | 내용 | 
| --- | --- |
| 0 | 포괄손익계산서 (연간) |
| 1 | 포괄손익계산서 (분기) |
| 2 | 재무상태표 (연간) |
| 3 | 재무상태표 (분기) |
| 4 | 현금흐름표 (연간) |
| 5 | 현금흐름표 (분기) |
```

In [11]:
from tqdm import tqdm
import time

quant_raw = pd.read_csv("./quant_raw.csv")
error_list = []

code_list = []
company_list = []
type_list = []
price_list = []
capital_list = []
GPA_list = []
GPA_q_list = []
ROE_list = []
ROE_q_list = []
ROA_list = []
ROA_q_list = []
PSR_list = []
PSR_q_list = []
PER_list = []
PER_q_list = []
PBR_list = []
PCR_list = []
PCR_q_list = []
NCAV_list = []
current_ratio_list = []
quick_ratio_list = []
debt_ratio_list = []
hae_list = []
ji_list = []
geum_list = []
spec_list = []
litz_list = []
manage_list = []
f1_list = []
f2_list = []
f3_list = []
f4_list = []
f5_list = []
f6_list = []
f7_list = []
f8_list = []
f9_list = []

for i in tqdm(range(0, kor_ticker.shape[0])):
    try:
        code = 'A'+kor_ticker.iloc[i]["종목코드"]

        if not (code in quant_raw["코드"].values):
            raise Exception("종목코드가 raw파일에 없습니다.")

        company = kor_ticker.iloc[i]["종목명"]
        type = "코스피" if kor_ticker.iloc[i]["시장구분"] == "KOSPI" else "코스닥"
        price = kor_ticker.iloc[i]["종가"]
        captial = kor_ticker.iloc[i]["시가총액"] / (10 ** 8)

        ticker = kor_ticker.iloc[i]["종목코드"]
        url = f'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{ticker}'
        data = pd.read_html(url, displayed_only=False)
        
        # 현재 시가총액 (억)
        market_capitalization = captial
        # 작년 매출총이익
        gross_fit_year = data[0][data[0]["IFRS(연결)"] == "매출총이익"].iloc[:,4].values[0]
        # 작년 총자산
        total_assets_year = data[2][data[2]["IFRS(연결)"] == "자산"].iloc[:,4].values[0]
        # 작년 당기순이익
        net_income_year = data[0][data[0]["IFRS(연결)"] == "당기순이익"].iloc[:,4].values[0]
        # 작년 총자본
        capital_year = data[2][data[2]["IFRS(연결)"] == "자본"].iloc[:,4].values[0]
        # 작년 매출액
        take_year = data[0][data[0]["IFRS(연결)"] == "매출액"].iloc[:,4].values[0]
        # 작년 영업활동현금흐름
        cash_flow = data[4][data[4]["IFRS(연결)"] == "영업활동으로인한현금흐름"].iloc[:,4].values[0]
        # 작년 부채 총계
        debt = data[2][data[2]["IFRS(연결)"] == "부채"].iloc[:,4].values[0]
        # 작년 유동자산
        current_assets = data[2][data[2]["IFRS(연결)"] == "유동자산계산에 참여한 계정 펼치기"].iloc[:,4].values[0]
        # 작년 당좌자산
        quick_assets_list = ["유동생물자산", "유동금융자산", "매출채권및기타유동채권", "당기법인세자산", "계약자산", "반품(환불)자산", "배출권", "기타유동자산", "현금및현금성자산", "매각예정비유동자산및처분자산집단"]
        quick_assets = 0
        for name in quick_assets_list:
            quick_assets += data[2][data[2]["IFRS(연결)"] == name].fillna(0).iloc[:,4].values[0]
        # 작년 유동부채
        current_liabilities = data[2][data[2]["IFRS(연결)"] == "유동부채계산에 참여한 계정 펼치기"].iloc[:,4].values[0]

        # 직전 분기 매출총이익
        gross_fit_quarter = data[1][data[1]["IFRS(연결)"] == "매출총이익"].iloc[:,4].values[0]
        # 직전 분기 총자산
        total_assets_quarter = data[3][data[3]["IFRS(연결)"] == "자산"].iloc[:,4].values[0]
        # 직전 분기 당기순이익
        net_income_quarter = data[1][data[1]["IFRS(연결)"] == "당기순이익"].iloc[:,4].values[0]
        # 직전 분기 총자본
        capital_quarter = data[3][data[3]["IFRS(연결)"] == "자본"].iloc[:,4].values[0]
        # 직전 분기 매출액
        take_quarter = data[1][data[1]["IFRS(연결)"] == "매출액"].iloc[:,4].values[0]
        # 직전 분기 영업활동현금흐름
        cash_flow_quarter = data[5][data[5]["IFRS(연결)"] == "영업활동으로인한현금흐름"].iloc[:,4].values[0]

        # GP/A
        GP_A = gross_fit_year / total_assets_year * 100
        # ROE
        ROE = net_income_year / capital_year * 100
        # ROA
        ROA = net_income_year / total_assets_year * 100
        # PSR
        PSR = market_capitalization / take_year
        # PER
        PER = market_capitalization / net_income_year
        # PBR
        PBR = market_capitalization / capital_year
        # PCR
        PCR = market_capitalization / cash_flow
        # 청산가치비율 (NCAV전략) (%)
        NCAV = (current_assets - debt) / market_capitalization * 100
        # 당좌비율
        quick_ratio = quick_assets / current_liabilities * 100
        # 유동비율
        current_ratio = current_assets / current_liabilities * 100
        # 부채비율
        debt_ratio = debt / capital_year * 100

        # 분기 GP/A
        GP_A_quarter = gross_fit_quarter * 4 / total_assets_quarter * 100
        GP_A_quarter = GP_A if np.isnan(GP_A_quarter) else GP_A_quarter
        # 분기 ROE
        ROE_quarter = net_income_quarter * 4 / capital_quarter * 100
        ROE_quarter = ROE if np.isnan(ROE_quarter) else ROE_quarter
        # 분기 ROA
        ROA_quarter = net_income_quarter * 4 / total_assets_quarter * 100
        ROA_quarter = ROA if np.isnan(ROA_quarter) else ROA_quarter
        # 분기 PSR
        PSR_quarter = market_capitalization / (take_quarter * 4)
        PSR_quarter = PSR if np.isnan(PSR_quarter) else PSR_quarter
        # 분기 PER
        PER_quarter = market_capitalization / (net_income_quarter * 4)
        PER_quarter = PER if np.isnan(PER_quarter) else PER_quarter
        # 분기 PCR
        PCR_quarter = market_capitalization / (cash_flow_quarter * 4)
        PCR_quarter = PCR if np.isnan(PCR_quarter) else PCR_quarter
        
        code_list.append(code)
        company_list.append(company)
        type_list.append(type)
        price_list.append(price)
        capital_list.append(captial)
        GPA_list.append(GP_A)
        GPA_q_list.append(GP_A_quarter)
        ROE_list.append(ROE)
        ROE_q_list.append(ROE_quarter)
        ROA_list.append(ROA)
        ROA_q_list.append(ROA_quarter)
        PSR_list.append(PSR)
        PSR_q_list.append(PSR_quarter)
        PER_list.append(PER)
        PER_q_list.append(PER_quarter)
        PBR_list.append(PBR)
        PCR_list.append(PCR)
        PCR_q_list.append(PCR_quarter)
        NCAV_list.append(NCAV)
        current_ratio_list.append(current_ratio)
        quick_ratio_list.append(quick_ratio)
        debt_ratio_list.append(debt_ratio)
        
        hae_list.append(quant_raw[quant_raw["코드"] == code]["해외 본사 =1"].values[0])
        ji_list.append(quant_raw[quant_raw["코드"] == code]["지주사 =1"].values[0])
        geum_list.append(quant_raw[quant_raw["코드"] == code]["금융 =1"].values[0])
        spec_list.append(quant_raw[quant_raw["코드"] == code]["스팩 =1"].values[0])
        litz_list.append(quant_raw[quant_raw["코드"] == code]["리츠 =1"].values[0])
        manage_list.append(quant_raw[quant_raw["코드"] == code]["관리종목 =1"].values[0])
        f1_list.append(quant_raw[quant_raw["코드"] == code]["F스코어 순이익>0 여부"].values[0])
        f2_list.append(quant_raw[quant_raw["코드"] == code]["F스코어 영업활동현금>0 여부"].values[0])
        f3_list.append(quant_raw[quant_raw["코드"] == code]["F스코어 ROA증가 여부"].values[0])
        f4_list.append(quant_raw[quant_raw["코드"] == code]["F스코어 영현>순익 여부"].values[0])
        f5_list.append(quant_raw[quant_raw["코드"] == code]["F스코어 부채비율감소 여부"].values[0])
        f6_list.append(quant_raw[quant_raw["코드"] == code]["F스코어 유동비율증가 여부"].values[0])
        f7_list.append(quant_raw[quant_raw["코드"] == code]["F스코어 신주발행X 여부"].values[0])
        f8_list.append(quant_raw[quant_raw["코드"] == code]["F스코어 매출총이익률증가 여부"].values[0])
        f9_list.append(quant_raw[quant_raw["코드"] == code]["F스코어 총자산회전율증가 여부"].values[0])
    except:
        error_list.append(code)

100%|██████████| 2683/2683 [17:00<00:00,  2.63it/s]


In [12]:
quant_data = pd.DataFrame({"코드" : code_list, "회사명" : company_list, "시장구분" : type_list,
                           "주가(원)" : price_list, "시가총액(억)" : capital_list, "GP/A (%)" : GPA_list, "분기 GP/A (%)" : GPA_q_list, "ROE (%)" : ROE_list,
                           "분기 ROE (%)" : ROE_q_list, "ROA (%)" : ROA_list, "분기 ROA (%)" : ROA_q_list, "PSR" : PSR_list, "분기 PSR" : PSR_q_list,
                           "PER" : PER_list, "분기 PER" : PER_q_list, "PBR" : PBR_list, "PCR" : PCR_list, "분기 PCR" : PCR_q_list, "청산가치비율 (NCAV전략) (%)" : NCAV_list,
                           "당좌 비율 (%)" : quick_ratio_list, "유동 비율 (%)" : current_ratio_list, "부채 비율 (%)" : debt_ratio_list, "해외 본사 =1" : hae_list,
                           "지주사 =1" : ji_list, "금융 =1" : geum_list, "스팩 =1" : spec_list, "리츠 =1" : litz_list, "관리종목 =1" : manage_list,
                           "F스코어 순이익>0 여부" : f1_list, "F스코어 영업활동현금>0 여부" : f2_list, "F스코어 ROA증가 여부" : f3_list, "F스코어 영현>순익 여부" : f4_list,
                           "F스코어 부채비율감소 여부" : f5_list, "F스코어 유동비율증가 여부" : f6_list, "F스코어 신주발행X 여부" : f7_list, "F스코어 매출총이익률증가 여부" : f8_list,
                           "F스코어 총자산회전율증가 여부" : f9_list})

In [13]:
quant_data.to_csv(f"./extraced_data_{biz_day}.csv",index=False,encoding="utf-8-sig")